In [23]:
datasheets = ["april", "august", "july", "may", "october", "september"]

import pandas as pd

In [51]:
import pandas as pd

ronis_data = pd.DataFrame()
for datasheet in datasheets:
    month_data = pd.read_csv(f"./data/{datasheet}_2024.csv")
    ronis_data = pd.concat([ronis_data, month_data], ignore_index=True)

In [54]:
ronis_data['Sent Date'] = pd.to_datetime(ronis_data['Sent Date']).dt.tz_localize('UTC')
ronis_data['unix_time'] = ronis_data['Sent Date'].astype(int) / 10**9

In [57]:
RONIS_LONGITUDE = "-96.3332428"
RONIS_LATITUDE = "30.6384402"

In [70]:
MIN_TIME, MAX_TIME = ronis_data["unix_time"].min(), ronis_data["unix_time"].max()
MIN_DATE, MAX_DATE = ronis_data["Sent Date"].min(), ronis_data["Sent Date"].max()

MIN_DATE, MAX_DATE = str(MIN_DATE)[:10], str(MAX_DATE)[:10]

API_KEY = "479ed8e6bd3a3932b6818fc89a371bc7"

print(MIN_TIME, MAX_TIME)

-9223372036.854776 1730411160.0


### Getting the historical weather data

In [103]:
hourly_dataframe = pd.read_csv("./data/weather_data.csv")

In [106]:
hourly_dataframe = hourly_dataframe.drop(columns=["Unnamed: 0"])

In [111]:
ronis_data["hour_date"] = ronis_data["Sent Date"].apply(lambda x: x.replace(minute=0, second=0))

In [117]:
hourly_dataframe[hourly_dataframe["date"] == "2024-04-13 11:00:00+00:00"]["temperature_2m"].values[0]

13.45

In [129]:
ronis_data["temperature"] = ronis_data["hour_date"].apply(
    lambda date: hourly_dataframe[hourly_dataframe["date"].astype(str) == str(date)]["temperature_2m"].values[0]
    if not hourly_dataframe[hourly_dataframe["date"].astype(str) == str(date)]["temperature_2m"].empty
    else None
)

In [132]:
ronis_data = ronis_data.drop(columns=["weather_2m"])

In [133]:
ronis_data

,Order #,Sent Date,Modifier,Option Group Name,Parent Menu Selection,Order ID,unix_time,hour_date,temperature
0,18,2024-04-13 11:34:00+00:00,Regular,Noods,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45
1,18,2024-04-13 11:34:00+00:00,Cheddar,Choose Your Cheese,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45
2,18,2024-04-13 11:34:00+00:00,Pulled Pork,Choose Your Meats,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45
3,18,2024-04-13 11:34:00+00:00,Broccoli,Choose Your Toppings,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45
4,18,2024-04-13 11:34:00+00:00,Tomatoes,Choose Your Toppings,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45
...,...,...,...,...,...,...,...,...,...
252723,3001,2024-09-30 21:00:00+00:00,Mushrooms,Choose Your Toppings,Mac and Cheese,30-3001,1.727730e+09,2024-09-30 21:00:00+00:00,34.35
252724,3001,2024-09-30 21:00:00+00:00,Parmesan,Choose Your Toppings,Mac and Cheese,30-3001,1.727730e+09,2024-09-30 21:00:00+00:00,34.35
252725,3001,2024-09-30 21:00:00+00:00,Garlic Parmesan,Choose Your Drizzles,Mac and Cheese,30-3001,1.727730e+09,2024-09-30 21:00:00+00:00,34.35
252726,3001,2024-09-30 21:00:00+00:00,No Side,Choose Your Side,Mac and Cheese,30-3001,1.727730e+09,2024-09-30 21:00:00+00:00,34.35


In [137]:
ronis_data["hour"] = ronis_data["Sent Date"].apply(lambda x: x.hour)
ronis_data["month"] = ronis_data["Sent Date"].apply(lambda x: x.month)

In [259]:
# meats, toppings, drizzles, cheese, drink, noods

meats = ronis_data[ronis_data["Option Group Name"] == "Choose Your Meats"]["Modifier"].unique()
toppings = ronis_data[ronis_data["Option Group Name"] == "Choose Your Toppings"]["Modifier"].unique()
drizzles = ronis_data[ronis_data["Option Group Name"] == "Choose Your Drizzles"]["Modifier"].unique()
cheese = ronis_data[ronis_data["Option Group Name"] == "Choose Your Cheese"]["Modifier"].unique()
drink = ronis_data[ronis_data["Option Group Name"] == "Choose Your Drink"]["Modifier"].unique()
noods = ronis_data[ronis_data["Option Group Name"] == "Noods"]["Modifier"].unique()


['Pulled Pork' 'Brisket' 'No Meat' 'Bacon' 'Grilled Chicken' 'Ham']
['Broccoli' 'Tomatoes' 'Breadcrumbs' 'Corn' 'Mushrooms' 'Parmesan'
 'Jalapenos' 'No Toppings' 'Bell Peppers' 'Pineapple' 'Onions']
['BBQ' 'No Drizzle' 'Garlic Parmesan' 'Hot Honey' 'Ranch' 'Buffalo'
 'Pesto']
['Cheddar' 'Pepper Jack' 'Alfredo' 'MIX' 'NO CHEESE' 'Gouda']


### figuring out if a day and month falls under a holiday

In [150]:
import json

with open('./data/holidays.json', 'r') as file:
    holidays = json.load(file)

In [154]:
split_holidays = []

for holiday in holidays:
    holiday["month"] = pd.to_datetime(holiday["date"]).month
    holiday["day"] = pd.to_datetime(holiday["date"]).day

    split_holidays.append(holiday)

In [178]:
holidays_array = []

for holiday in split_holidays:
    holidays_array.append(f"{holiday['month']}-{holiday['day']}")

def classify_holiday(date):
    month = date.month
    day = date.day 

    date_string = f"{month}-{day}"

    return 1 if date_string in holidays_array else 0

def classify_weekend(date):
    return 1 if date.weekday() >= 5 else 0

In [179]:
ronis_data['holiday'] = ronis_data.groupby('Order #')['Sent Date'].transform(
    lambda x: x.apply(classify_holiday)
)

ronis_data['weekend'] = ronis_data.groupby('Order #')['Sent Date'].transform(
    lambda x: x.apply(classify_weekend)
)

In [262]:
ronis_data

,Order #,Sent Date,Modifier,Option Group Name,Parent Menu Selection,Order ID,unix_time,hour_date,temperature,hour,month,holiday,weekend
0,18,2024-04-13 11:34:00+00:00,Regular,Noods,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45,11.0,4.0,1,1
1,18,2024-04-13 11:34:00+00:00,Cheddar,Choose Your Cheese,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45,11.0,4.0,1,1
2,18,2024-04-13 11:34:00+00:00,Pulled Pork,Choose Your Meats,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45,11.0,4.0,1,1
3,18,2024-04-13 11:34:00+00:00,Broccoli,Choose Your Toppings,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45,11.0,4.0,1,1
4,18,2024-04-13 11:34:00+00:00,Tomatoes,Choose Your Toppings,Mac and Cheese,13-18,1.713008e+09,2024-04-13 11:00:00+00:00,13.45,11.0,4.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252723,3001,2024-09-30 21:00:00+00:00,Mushrooms,Choose Your Toppings,Mac and Cheese,30-3001,1.727730e+09,2024-09-30 21:00:00+00:00,34.35,21.0,9.0,0,0
252724,3001,2024-09-30 21:00:00+00:00,Parmesan,Choose Your Toppings,Mac and Cheese,30-3001,1.727730e+09,2024-09-30 21:00:00+00:00,34.35,21.0,9.0,0,0
252725,3001,2024-09-30 21:00:00+00:00,Garlic Parmesan,Choose Your Drizzles,Mac and Cheese,30-3001,1.727730e+09,2024-09-30 21:00:00+00:00,34.35,21.0,9.0,0,0
252726,3001,2024-09-30 21:00:00+00:00,No Side,Choose Your Side,Mac and Cheese,30-3001,1.727730e+09,2024-09-30 21:00:00+00:00,34.35,21.0,9.0,0,0


In [231]:
ronis_meat_df = ronis_data[ronis_data["Option Group Name"] == "Choose Your Meats"]
ronis_toppings_df = ronis_data[ronis_data["Option Group Name"] == "Choose Your Toppings"]
ronis_drizzles_df = ronis_data[ronis_data["Option Group Name"] == "Choose Your Drizzles"]
ronis_cheese_df = ronis_data[ronis_data["Option Group Name"] == "Choose Your Cheese"]

In [271]:
def get_training_data(df, unique_items):
    unique_hours = df["hour_date"].unique()
    all_items_info = []

    for hour in unique_hours:
        hour_item_info = {}
        hour_item_info["month"] = pd.to_datetime(hour).month
        hour_item_info["day"] = pd.to_datetime(hour).day
        hour_item_info["hour"] = pd.to_datetime(hour).hour
        
        for item in unique_items:
            hour_item_info[item] = 0

        hourly_group = df[df["hour_date"] == hour]

        for index, row in hourly_group.iterrows():
            row_item = row["Modifier"]
            hour_item_info[row_item] += 1
            hour_item_info["temperature"] = row["temperature"]
            hour_item_info["holiday"] = row["holiday"]
            hour_item_info["weekend"] = row["weekend"]

        all_items_info.append(hour_item_info)


    ronis_hourly_data = pd.DataFrame(all_items_info)

    return ronis_hourly_data

In [272]:
ronis_topping_trainable_data = get_training_data(ronis_toppings_df, toppings)
ronis_meat_training_data = get_training_data(ronis_meat_df, meats)
ronis_drizzles_training_data = get_training_data(ronis_drizzles_df, drizzles)
ronis_cheese_training_data = get_training_data(ronis_cheese_df, cheese)

In [273]:
ronis_meat_training_data

,month,day,hour,Pulled Pork,Brisket,No Meat,Bacon,Grilled Chicken,Ham,temperature,holiday,weekend
0,4,13,11,3,12,2,4,3,0,13.45,1,1
1,4,13,12,2,15,8,1,11,2,13.40,1,1
2,4,13,13,3,12,3,5,4,0,15.00,1,1
3,4,13,14,3,7,7,4,9,0,17.50,1,1
4,4,13,15,1,12,4,5,4,0,20.00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1685,9,30,17,0,5,5,0,9,0,31.90,0,0
1686,9,30,18,0,9,1,6,6,2,32.85,0,0
1687,9,30,19,0,10,10,9,12,2,33.65,0,0
1688,9,30,20,0,7,2,2,8,1,34.15,0,0


In [240]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler

import joblib

In [274]:
def train_model(df, outputs):
    # Create a copy to avoid modifying the original dataframe
    normalized_data = df.copy()

# For each row, find the max meat value and set it to 1, others to 0
    for idx in normalized_data.index:
        item_values = normalized_data.loc[idx, outputs]
        max_value = item_values.max()
        normalized_data.loc[idx, outputs] = (item_values == max_value).astype(int)

    X = normalized_data[['day', 'month', 'hour', 'temperature', 'holiday', 'weekend']]
    y = normalized_data[outputs]


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    model = MultiOutputClassifier(RandomForestClassifier(random_state=42))
    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)


    return model

In [252]:
def save_model(model, name):
    joblib.dump(model, f"./models/{name}.joblib")

In [275]:
meat_model = train_model(ronis_meat_training_data, meats)
toppings_model = train_model(ronis_topping_trainable_data, toppings)
drizzles_model = train_model(ronis_drizzles_training_data, drizzles)
cheese_model = train_model(ronis_cheese_training_data, cheese)

In [276]:
save_model(meat_model, "meat_prediction")
save_model(toppings_model, "toppings_prediction")
save_model(drizzles_model, "drizzles_prediction")
save_model(cheese_model, "cheese_prediction")